# 차수 효율화 프로젝트

### 목표 

차수 배분에 대한 최적 효율화를 통해 더 높은 생산성 유도. 

### 얻는 이점

1. 1차수에 들어갈 최적의 주문건수 의사결정을 내리도록 도움
2. DAS를 이용하지 않아도 되는 주문은 배제시킴


## 문제

차수를 어떻게 효율적으로 나눌것인가가 문제. <br>
90개의 Cells 이 있다고 가정을 했을때, 90개의 주문을 1차수에 넣으면 모든 물품은 문제없이 소화 가능<br>
하지만 1차수에 90개 이상의 주문을 넣었을때 더 효율적으로 상품을 가져올수 있는 기회를 잃어버림. 

1차수에 너무 많은 주문을 넣었다면 (예를 들어 200개의 주문을 1차수에 넣음) DAS 분배과정에서 <br>
Cells있지 않은 상품을 선택하여 생기는 비효율성이 생길수 있음. 

결국 그 둘 사이의 최적점을 찾아서 1차수에 들어갈수 있는 최적의 "주문" 갯수를 알아내는 것이 중요. 

## 알고리즘 


### 주문간의 유사도 판별

문제의 유사도 판별은 probability model 그리고 sigmoid activation을 사용함. <br>

$$ H(A, B) = \frac{1}{1+e^{-\sum A주문 * B주문}} $$ 

예를 들어 주문 A에는 a상품 3개, b2개 라는 상품을 갖고 있고, <br>
B라는 주문애는 a상품 3개, b상품 2개, c상품 1개 라는 상품을 갖고 있으며, <br>
C라는 주문에는 c상품 1개, d상품 2개를 갖고 있음 

$$ \begin{align}
A &= \{ 3a, 2b \} \\ 
B &= \{ 2a, 2b, 1c \} \\
C &= \{ 1c, 2d \}
\end{align} $$ 

위의 주문은 선형대수학에서 말하는 vector의 형태로 전환이 가능. 

$$ \begin{align}
A &= [ 3, 2, 0, 0] \\ 
B &= [ 2, 2, 1, 0] \\
C &= [ 0, 0, 1, 2]
\end{align} $$ 

vectors로 표현된 주문들은 서로간의 유사도 판별이 가능해지며 유사도 수치에 따라서 어떤 상품을 같은 차수에 넣을지 결정을 할 수 있음. <br>
아래는 서로 유사도를 비교하는 예제 코드. 

즉 결론적으로 서로 주문들간의 유사도가 높으면 높을수록 1에 가까워지는 확률을 보이며, <br>
서로 상관관계가 전혀 없을시에는 0에 가까운 값을 내놓음

아래의 예제에서는 A와 B의 유사도 0.809는 A 와 C의 유사도 0.5값보다 높다. 

In [58]:
import numpy as np

MAXIMUM = 3

def sigmoid(x):
    return 1/(1 + np.nan_to_num(np.e**(-x)))


def forward(a, b):
    a = a/MAXIMUM
    b = b/MAXIMUM
    return np.sum(a * b)

def process(a, b):
    return sigmoid(forward(a, b))

A = np.array([3, 2, 0, 0])
B = np.array([3, 2, 1, 0])
C = np.array([0, 0, 1, 2])

r1 = process(A, B) # 0.999997739676
r2 = process(A, C) # 0.5
r3 = process(B, C) # 0.527749235055


print('H(A, B) =', r1)
print('H(A, C) =', r2)
print('H(B, C) =', r3)

H(A, B) = 0.809141955755
H(A, C) = 0.5
H(B, C) = 0.527749235055


### 우선순위 문제

셀의 전체 갯수가 4개라고 가정을 하고, 현재 모두 다 가득차 있는 상태라면 다음과 같이 그려질수 있음 

$$
\text{1번 셀} = [4, 2, 0, 0] \\
\text{2번 셀} = [3, 2, 0, 0] \\
\text{3번 셀} = [0, 0, 1, 0] \\
\text{4번 셀} = [1, 0, 2, 0]
$$

현재 DAS상태를 대표할수 있는 값은 모든 cell vectors를 합한 값. 즉.. 

$$ \text{1번 셀} + \text{2번 셀} + \text{3번 셀} + \text{4번 셀} = [8, 4, 3, 0] $$



이 때 어떤 주문을 넣는것이 (같은 차수안에..) 가장 효율적인가에 대한 문제가 생김<br>
예를 들어서 D, E, F라는 선택 가능한 주문이 있음
(현재 Cell의 4칸 모두 주문이 차 있는 상태) 

$$ \begin{align}
D &= [3, 3, 1, 0 ] \\ 
E &= [0, 0, 1, 5 ] \\
F &= [0, 0, 0, 5 ] \\
\end{align} $$ 


아래의 예제 코드에서 나온봐와 같이, 유사도 판별을 하여 가장 유사한 상품을 차수에 집어 넣으면 됨

In [62]:
R = np.array([8, 4, 3, 2])
D = np.array([3, 3, 1, 0])
E = np.array([0, 0, 1, 5])
F = np.array([0, 0, 0, 5])

print('H(R, D) =', process(R, D))
print('H(R, E) =', process(R, E))
print('H(R, F) =', process(R, F))

H(R, D) = 0.987046272469
H(R, E) = 0.809141955755
H(R, F) = 0.752336198861



### 1차수에 얼만큼 더 넣을지 문제 

사실 정답이 없음. <br>
정답을 가장 잘 아는 사람은 바로 현장에서 일하고 계신 분들이 가장 잘 알고 있음. <br>
즉 현장 사람들이 정할수 있도록 옵션을 주는 것이 가장 효율적이라고 판단. 

**즉 해당 비율을 현장에 계신 분들이 정하면 그에 따라서 자동으로 차수를 나눠줌**

$$ \text{비율} = \frac{\text{못들어가는 양}}{\text{들어갈수 있는 양}} $$

비율을 정한는 방법은 몬테 카를로 알고리즘을 사용하여 가상의 시뮬레이션을 돌려본뒤 결정이 됨 

가장 낮은 주문건부터 
A 라는 상품이 10개 
1번 주분 A, B
2번 주문 A, B, C 상품
3번 주문 A, B, C, D

주문상품이 적은 주문부터 